In [3]:
import pandas as pd

# Load the data
liaocheng_air_quality = pd.read_csv('../랴오청대기질.csv', encoding='cp949')
seoul_weather = pd.read_csv('../서울날씨.csv', encoding='cp949')
seoul_air_quality = pd.read_csv('../서울대기질.csv', encoding='cp949')

# Replace blank spaces with NaN
liaocheng_air_quality.replace(' ', pd.NA, inplace=True)
seoul_air_quality.replace(' ', pd.NA, inplace=True)

# Convert date columns to datetime format
liaocheng_air_quality['date'] = pd.to_datetime(liaocheng_air_quality['date'])
seoul_air_quality['date'] = pd.to_datetime(seoul_air_quality['date'], format='%Y/%m/%d')
seoul_weather['일시'] = pd.to_datetime(seoul_weather['일시'])

seoul_weather.fillna(0, inplace=True)

# Convert other columns to numeric
liaocheng_air_quality.iloc[:, 1:] = liaocheng_air_quality.iloc[:, 1:].apply(pd.to_numeric)
seoul_air_quality.iloc[:, 1:] = seoul_air_quality.iloc[:, 1:].apply(pd.to_numeric)


In [ ]:
# Merge air quality data on date
combined_data = pd.merge(liaocheng_air_quality, seoul_air_quality, on='date', suffixes=('_liaocheng', '_seoul'))

# Merge the combined data with weather data
final_data = pd.merge(combined_data, seoul_weather, left_on='date', right_on='일시')

# Drop the duplicate date column
final_data.drop(columns=['일시'], inplace=True)

# Display the combined data
final_data.head()


In [15]:
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score

# NaN 값을 앞의 값으로 채우기
final_data.fillna(method='ffill', inplace=True)

# SARIMA 모델 적합
model = SARIMAX(final_data[' pm25_seoul'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), 
                exog=final_data[[' pm25_liaocheng', '평균기온(℃)', '평균풍속(m/s)', '강수량(mm)']])
results = model.fit()

# 예측할 기간 설정 (사용자가 선택)
forecast_steps = 30  # 예측할 기간을 일수로 지정 (여기서는 30일로 예시)
forecast = results.get_forecast(steps=forecast_steps, exog=final_data[[' pm25_liaocheng', '평균기온(℃)', '평균풍속(m/s)', '강수량(mm)']].iloc[-forecast_steps:])
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# 결정계수(R²) 계산
actual_values = final_data[' pm25_seoul'].iloc[-forecast_steps:]
r_squared = r2_score(actual_values, predicted_mean)

# 결정계수 출력
print(f'결정계수 (R²): {r_squared:.4f}')

# 예측된 값 표시
predicted_mean.head()


C:\Users\SUB\AppData\Local\Temp\ipykernel_13900\3515970048.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_data.fillna(method='ffill', inplace=True)


결정계수 (R²): -0.0942


2096    50.901398
2097    63.412643
2098    68.264359
2099    61.391255
2100    69.679958
Name: predicted_mean, dtype: float64

In [22]:
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

# NaN 값을 앞의 값으로 채우기
final_data.fillna(method='ffill', inplace=True)

# 예측할 특정 날짜와 예측 기간을 입력 받기
predict_from_date = '2023-07-01'  # 예측 시작 날짜 (예시: 2023년 7월 1일)
predict_from_date = pd.to_datetime(predict_from_date)
forecast_days = 30  # 예측할 일수 (예시: 30일)

# 특정 날짜까지의 데이터만 사용
train_data = final_data[final_data['date'] <= predict_from_date]

# SARIMA 모델 적합
model = SARIMAX(train_data[' pm25_seoul'], order=(1, 1, 1), seasonal_order=(1, 1, 1, 12), 
                exog=train_data[[' pm25_liaocheng', '평균기온(℃)', '평균풍속(m/s)', '강수량(mm)']])
results = model.fit()

# 특정 날짜 이후의 데이터로 예측
exog_future = final_data[final_data['date'] > predict_from_date][[' pm25_liaocheng', '평균기온(℃)', '평균풍속(m/s)', '강수량(mm)']].iloc[:forecast_days]
if exog_future.shape[0] < forecast_days:
    raise ValueError(f"외생 변수 데이터가 충분하지 않습니다. 필요한 데이터 수: {forecast_days}, 실제 데이터 수: {exog_future.shape[0]}")

forecast = results.get_forecast(steps=forecast_days, exog=exog_future)
predicted_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

# 결정계수(R²) 계산 (예측 구간 내 실제값이 존재하는 경우에만 계산)
if len(final_data[' pm25_seoul'].iloc[train_data.index[-1] + 1: train_data.index[-1] + 1 + forecast_days]) == forecast_days:
    actual_values = final_data['pm25_seoul'].iloc[train_data.index[-1] + 1: train_data.index[-1] + 1 + forecast_days]
    r_squared = r2_score(actual_values, predicted_mean)
    print(f'결정계수 (R²): {r_squared:.4f}')
else:
    print('결정계수를 계산할 실제값 데이터가 부족합니다.')

# 예측된 값 표시
predicted_mean.head()


C:\Users\SUB\AppData\Local\Temp\ipykernel_13900\1121983563.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  final_data.fillna(method='ffill', inplace=True)
c:\ProgramData\anaconda3\envs\py310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\envs\py310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
